In [2]:
import pandas as pd

In [73]:
def asset_cash_flow(principle, wac, wam, prepay_rate, annual_default, default_loss):
    balance = principle
    month = 1
    remaining = wam 
    asset_balance = []
    scheduled_payment = []
    scheduled_interest = []
    scheduled_principle = []
    prepaid_principle = [] 
    defaulted_principle = [] 
    total_principle = []
    total_payment = [] 
    loss = []
    
    for month in range(wam):
        payment = (balance * wac/12)/(1-(1/(1+wac/12)**(remaining)))
        interest = (balance * wac / 12)
        principle = payment - interest
        prepaid = balance * prepay_rate / 12 
        default = balance * annual_default / 12 
        asset_balance.append(balance)
        scheduled_payment.append(payment)
        scheduled_interest.append(interest)
        scheduled_principle.append(principle)
        prepaid_principle.append(prepaid)
        defaulted_principle.append(default)
        total_principle.append(principle + prepaid + (1 - default_loss) * default )
        total_payment.append(principle + prepaid + (1 - default_loss) * default + interest)
        loss.append(default_loss * default)
        balance = balance - principle - prepaid - default
        month = month + 1
        remaining = remaining -1
    if wam < 360:
        count = 360 - wam
        i = 1
        for i in range(count):
            asset_balance.append(0)
            scheduled_payment.append(0)
            scheduled_interest.append(0)
            scheduled_principle.append(0)
            prepaid_principle.append(0)
            defaulted_principle.append(0)
            total_principle.append(0)
            total_payment.append(0)
            loss.append(0)
    d = {'Balance': asset_balance, 'Scheduled Payment': scheduled_payment,'Interest':scheduled_interest, 'Scheduled Principle':scheduled_principle, 'Prepaid Principle':prepaid_principle,'Defaulted Principle':defaulted_principle,'Total Principle':total_principle,'Total Payment':total_payment,'Loss':loss}
    df = pd.DataFrame(d).round(2)
    df.index += 1
    return df 

def note_interest(balance, libor, spread):
    interest = balance * (libor + spread)/4
    return interest


def note_price(note, rate):
    price = 0
    rate = rate/4
    for i in range(1,121):
        price = price + (note.loc[i,'Interest'] + note.loc[i,'Principle'])/(1+rate)**i
    return price.round(2)

def average_life(note):
    neumerator = 0
    denominator = 0 
    for i in range(1,121):
        principle = note.loc[i,'Principle']
        neumerator = neumerator + (i * principle)
        denominator = denominator + principle
    return neumerator/denominator

In [82]:
asset_a = asset_cash_flow(103000000,0.0625,342,0.05,0.1,0.4)
asset_b = asset_cash_flow(103000000,0.0702,351,0.02,0.1,0.3)
asset_c = asset_cash_flow(102000000,0.0547,343,0.10,0.1,0.3)
asset_d = asset_cash_flow(101000000,0.0602,351,0.05,0.5,0.1)
asset_e = asset_cash_flow(101000000,0.0817,360,0.05,0.25,0.25)
quarterly_principle = 0
quarterly_interest = 0
note_a_interest = []
note_a_principle = []
note_b_interest = []
note_b_principle = []
note_c_interest = []
note_c_principle = []
note_a_balance = 325000000
note_b_balance = 125000000
note_c_balance = 50000000
note_a_balance_tracker = []
note_b_balance_tracker = []
note_c_balance_tracker = []
note_a_ratio = 0.65
note_b_ratio = 0.25
note_c_ratio = 0.1
total_loss = 0
month_36_loss = 0
month = 1

excess = 0

for quarter in range(120): #Primary loop that counts by quarter for a total of 120 quarters. 
    note_a_balance_tracker.append(note_a_balance)
    note_b_balance_tracker.append(note_b_balance)
    note_c_balance_tracker.append(note_c_balance)
    #Reset the principle and interest holders to zero at the start of each quarter    
    quarterly_principle = 0 
    quarterly_interest = 0
    for count in range(3):  #Loop that gathers the monthly payments and loss from assets and stores the values for the quarter. 
        quarterly_interest = quarterly_interest * (1 + 0.03/12) + asset_a.loc[month,'Interest'] + asset_b.loc[month,'Interest'] + asset_c.loc[month,'Interest'] + asset_d.loc[month,'Interest'] + asset_e.loc[month,'Interest']
        quarterly_principle = quarterly_principle * (1 + 0.03/12) + asset_a.loc[month,'Total Principle'] + asset_b.loc[month,'Total Principle'] + asset_c.loc[month,'Total Principle'] + asset_d.loc[month,'Total Principle'] + asset_e.loc[month,'Total Principle']
        total_loss = total_loss + asset_a.loc[month,'Loss'] + asset_b.loc[month,'Loss'] + asset_c.loc[month,'Loss'] + asset_d.loc[month,'Loss'] + asset_e.loc[month,'Loss']
        month = month + 1
    if month == 37:
        month_36_loss = total_loss
    #Begin distribuition of cash flowes to the notes.       
    if month <= 37:  #Conditional statement to perform logic for months 1 through 36 lock out. 
         #calculates the required interst for each note, tests against the collected cash flows, and distributes in order of priority
        required_interest_a = note_interest(note_a_balance, 0.04, 0.0125) #Note A
        if quarterly_interest >= required_interest_a:
            note_a_interest.append(required_interest_a)
            quarterly_interest = quarterly_interest - required_interest_a
        else:
            note_a_interest.append(quarterly_interest)
            quarterly_interest = 0
            
        required_interest_b = note_interest(note_b_balance, 0.04, 0.03) #Note B
        if quarterly_interest >= required_interest_b:
            note_b_interest.append(required_interest_b)
            quarterly_interest = quarterly_interest - required_interest_b
        else:
            note_b_interest.append(quarterly_interest)
            quarterly_interest = 0
            
        required_interest_c = note_interest(note_c_balance, 0.04, 0.1) #Note C
        if quarterly_interest >= required_interest_c:
            note_c_interest.append(required_interest_c)
            quarterly_interest = quarterly_interest - required_interest_c
            excess = excess * (1 + 0.03/4) + quarterly_interest  #Any excess interest is deposited into excess account which earns interest at LIBOR - 100BP. 
        else:
            note_c_interest.append(quarterly_interest)
        #Principle only distributed to note A during lockout
        note_a_principle.append(quarterly_principle)
        note_a_balance = note_a_balance - quarterly_principle
        note_b_principle.append(0) #Value of 0 stored for nores B and C principle payments during lockout. 
        note_c_principle.append(0)
    #Test for post lock out loss
    elif month_36_loss < 50000000:
         #calculates the required interst for each note, tests against the collected cash flows, and distributes in order of priority
        #Note A
        required_interest_a = note_interest(note_a_balance, 0.04, 0.0125) 
        if quarterly_interest >= required_interest_a:
            note_a_interest.append(required_interest_a)
            quarterly_interest = quarterly_interest - required_interest_a
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_a - quarterly_interest) <= excess:
            note_a_interest.append(required_interest_a)
            excess = excess - (required_interest_a - quarterly_interest)
            quarterly_interest = 0
        else:
            note_a_interest.append(quarterly_interest + excess)
            quarterly_interest = 0
            excess = 0 
        #Note B    
        required_interest_b = note_interest(note_b_balance, 0.04, 0.03) 
        if quarterly_interest >= required_interest_b:
            note_b_interest.append(required_interest_b)
            quarterly_interest = quarterly_interest - required_interest_b
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_b - quarterly_interest) <= excess:
            note_b_interest.append(required_interest_b)
            excess = excess - (required_interest_b - quarterly_interest)
            quarterly_interest = 0
        else:
            note_b_interest.append(quarterly_interest + excess)
            quarterly_interest = 0
            excess = 0 
        #Note C
        required_interest_c = note_interest(note_c_balance, 0.04, 0.1) 
        if quarterly_interest >= required_interest_c:
            note_c_interest.append(required_interest_c)
            quarterly_interest = quarterly_interest - required_interest_c
            excess = excess + quarterly_interest  #Any excess interest is deposited into excess account. 
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_c - quarterly_interest) <= excess:
            note_c_interest.append(required_interest_c)
            excess = excess - (required_interest_c - quarterly_interest)
        else:
            note_c_interest.append(quarterly_interest + excess)
            excess = 0 
        #Principle after lockout
        #calculate pro rata share of principle to notes A and B 
        note_a = quarterly_principle * note_a_ratio
        note_b = quarterly_principle * note_b_ratio
        note_c = quarterly_principle * note_c_ratio
        #tests that note A principle balance is greater than the pro rata share of principle allocated to note A
        #if not, note A is retired
        if (note_a_balance < note_a):
            note_a_principle.append(note_a_balance)
            quarterly_principle = quarterly_principle - note_a_balance
            note_a_balance = 0
            #reset note ratios
            note_a_ratio = 0
            note_b_ratio = 125/175
            note_c_ratio = 50/175
            
            #recalculate note B and C to distribute excess principle from note A
            note_b = quarterly_principle * note_b_ratio
            note_c = quarterly_principle * note_c_ratio
            
        else: #normal principle allocation to note A
            note_a_principle.append(note_a)
            note_a_balance = note_a_balance - note_a
        
        #tests that note B principle balance is greater than the pro rata share of principle allocated to note B
        #if not, note B is retired
        if (note_b_balance < note_b):
            note_b_principle.append(note_b_balance)
            quarterly_principle = quarterly_principle - note_b_balance
            note_b_balance = 0
            #reset note ratios
            note_a_ratio = 0
            note_b_ratio = 0
            note_c_ratio = 1
            #recalculate note C to distribute excess principle from note B
            note_c = quarterly_principle * note_c_ratio
            
        else: #normal principle allocation to note B
            note_b_principle.append(note_b)
            note_b_balance = note_b_balance - note_b
        #note C receives principle
        note_c_principle.append(note_c)
        note_c_balance = note_c_balance - note_c
    else: #code for after lock out with loss greater than $50,000,000
        #calculates the required interest for each note, tests against the collected cash flows, and distributes in order of priority
        #Note A
        required_interest_a = note_interest(note_a_balance, 0.04, 0.0125) 
        if quarterly_interest >= required_interest_a:
            note_a_interest.append(required_interest_a)
            quarterly_interest = quarterly_interest - required_interest_a
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_a - quarterly_interest) <= excess:
            note_a_interest.append(required_interest_a)
            excess = excess - (required_interest_a - quarterly_interest)
            quarterly_interest = 0
        else:
            note_a_interest.append(quarterly_interest + excess)
            quarterly_interest = 0
            excess = 0 
        #Note B    
        required_interest_b = note_interest(note_b_balance, 0.04, 0.03) 
        if quarterly_interest >= required_interest_b:
            note_b_interest.append(required_interest_b)
            quarterly_interest = quarterly_interest - required_interest_b
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_b - quarterly_interest) <= exccess:
            note_b_interest.append(required_interest_b)
            excess = excess - (required_interest_b - quarterly_interest)
            quarterly_interest = 0
        else:
            note_b_interest.append(quarterly_interest + excess)
            quarterly_interest = 0
            excess = 0 
        #Note C
        required_interest_c = note_interest(note_c_balance, 0.04, 0.1) 
        if quarterly_interest >= required_interest_c:
            note_c_interest.append(required_interest_c)
            quarterly_interest = quarterly_interest - required_interest_c
            excess = excess + quarterly_interest  #Any excess interest is deposited into excess account. 
        #Excess interest account can be used to pay any interest deficiencies to notes by order of priority. 
        elif (required_interest_c - quarterly_interest) <= excess:
            note_c_interest.append(required_interest_c)
            excess = excess - (required_interest_c - quarterly_interest)
        else:
            note_c_interest.append(quarterly_interest + excess)
            excess = 0 
            
        #principle calculations
        if note_a_balance > 0:
            if note_a_balance < quarterly_principle:
                note_a_principle.append(note_a_balance)
                quarterly_principle = quarterly_principle - note_a_balance
                note_a_balance = 0
                note_b_principle.append(quarterly_principle)
                note_b_balance = note_b_balance - quarterly_principle
                note_c_principle.append(0)
            else: 
                note_a_principle.append(quarterly_principle)
                note_b_principle.append(0)
                note_c_principle.append(0)
        elif note_b_balance > 0:
            if note_b_balance < quarterly_principle:
                note_b_principle.append(note_b_balance)
                quarterly_principle = quarterly_principle - note_b_balance
                note_b_balance = 0
                note_c_principle.append(quarterly_principle)
                note_c_balance = note_b_balance - quarterly_principle
            else:
                note_b_principle.append(quarterly_principle)
                note_c_principle.append(0)
        else:
            note_c_principle.append(quarterly_principle)
            
#uses excess account to payoff any principle on note A at the end of month 360.
if note_a_balance > 0 and excess > 0:
    if note_a_balance - excess > 0:
        last =  note_a_principle.pop()
        note_a_principle.append(last + excess)
        note_a_balance = note_a_balance - excess
        note_a_balance_tracker.pop()
        note_a_balance_tracker.append(note_a_balance)
        excess = 0
    else: 
        note_a_principle.pop()
        note_a_principle.append(note_a_balance)
        excess = excess - note_a_balance
        note_a_balance = 0
        note_a_balance_tracker.pop()
        note_a_balance_tracker.append(0)
        
#uses excess account to payoff any principle on note B at the end of month 360.
if note_b_balance > 0 and excess > 0:
    if note_b_balance - excess > 0:
        last =  note_b_principle.pop()
        note_b_principle.append(last + excess)
        note_b_balance = note_b_balance - excess
        note_b_balance_tracker.pop()
        note_b_balance_tracker.append(note_b_balance)
        excess = 0
    else: 
        note_b_principle.pop()
        note_b_principle.append(note_b_balance)
        excess = excess - note_b_balance
        note_b_balance = 0
        note_b_balance_tracker.pop()
        note_b_balance_tracker.append(0)
        
#uses excess account to payoff any principle on note C at the end of month 360.
if note_c_balance > 0 and excess > 0:
    if note_a_balance - excess > 0:
        last =  note_c_principle.pop()
        note_c_principle.append(last + excess)
        note_c_balance = note_c_balance - excess
        note_c_balance_tracker.pop()
        note_c_balance_tracker.append(note_c_balance)
        excess = 0
    else: 
        note_c_principle.pop()
        note_c_principle.append(note_c_balance)
        excess = excess - note_c_balance
        note_c_balance = 0
        note_c_balance_tracker.pop()
        note_c_balance_tracker.append(0)
            
a = {'Balance':note_a_balance_tracker,'Interest': note_a_interest, 'Principle': note_a_principle}          
data_frame_note_a = pd.DataFrame(a).round(2)
data_frame_note_a.index += 1             

b = {'Balance':note_b_balance_tracker,'Interest': note_b_interest, 'Principle': note_b_principle}          
data_frame_note_b = pd.DataFrame(b).round(2)
data_frame_note_b.index += 1   

c = {'Balance':note_c_balance_tracker,'Interest': note_c_interest, 'Principle': note_c_principle}          
data_frame_note_c = pd.DataFrame(c).round(2)
data_frame_note_c.index += 1     



In [64]:
data_frame_note_a
#Displays data from note A

,Balance,Interest,Principle
1,3.250000e+08,4265625.00,28760514.01
2,2.962395e+08,3888143.25,26218731.08
3,2.700208e+08,3544022.41,23950294.36
4,2.460705e+08,3229674.79,21922437.42
5,2.241480e+08,2941942.80,20106532.85
6,2.040415e+08,2678044.56,18477556.54
7,1.855639e+08,2435526.63,17013622.06
8,1.685503e+08,2212222.84,15695576.02
9,1.528547e+08,2006218.41,14506646.12
10,1.383481e+08,1815818.68,13432135.49


In [65]:
data_frame_note_b
#Displays data from note B

,Balance,Interest,Principle
1,1.250000e+08,2187500.00,0.00
2,1.250000e+08,2187500.00,0.00
3,1.250000e+08,2187500.00,0.00
4,1.250000e+08,2187500.00,0.00
5,1.250000e+08,2187500.00,0.00
6,1.250000e+08,2187500.00,0.00
7,1.250000e+08,2187500.00,0.00
8,1.250000e+08,2187500.00,0.00
9,1.250000e+08,2187500.00,0.00
10,1.250000e+08,2187500.00,0.00


In [37]:
data_frame_note_c
#Displays data from note C

,Balance,Interest,Principle
1,50000000.00,1750000.00,0.00
2,50000000.00,1610857.69,0.00
3,50000000.00,1459856.09,0.00
4,50000000.00,1319784.46,0.00
5,50000000.00,1189682.36,0.00
6,50000000.00,1068687.59,0.00
7,50000000.00,956025.21,0.00
8,50000000.00,850997.49,0.00
9,50000000.00,752975.44,0.00
10,50000000.00,661391.34,0.00


In [46]:
#calculate price of note A
note_price(data_frame_note_a,0.0525)


325000000.03

In [47]:
#calculate price of note A
note_price(data_frame_note_b,0.07)

95775037.9

In [48]:
#calculate price of note A

note_price(data_frame_note_c,0.14)

20337876.4

In [80]:
#Calculate average life of note A
average_life(data_frame_note_a)

11.183167651619494

In [81]:
#Calculate average life of note B
average_life(data_frame_note_b)

41.52763092644862

In [79]:
#Calculate average life of note C
average_life(data_frame_note_c)

41.527630994986474